# Training a Linear Classifier on Spiral Data

We will be training a linear classifier, using a hinge loss, on our toy dataset of points. After training our model, we will be able to visualize the decision boundary that it learned!

Model:
- $f(X; W, b) = XW + B$

Loss:
- Hinge loss, no regularization

In [ ]:
import numpy as np

try:
    from jupyterthemes import jtplot
    jtplot.style()
except ImportError:
    pass

import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
from datasets import ToyData
from mygrad import Tensor
toy_data = ToyData()

Read the docstring for the `ToyData` class (place your cursor on `ToyData()` and hit shift-tab!). How many classes (tendrils) of data will be generated by the dataset?

### Loading the dataset
Look up the method for loading the data from the `toy_data` instance (`toy_data.<tab> brings up available attributes/methods`), and load in the training data/labels, and the testing data/labels. We actually will not be working with the testing data/labels for this notebook.

In [ ]:
xtrain, ytrain, xtest, ytest = toy_data.load_data()

### Learning about the data

What is the dimensionality of the training data? How many pieces of data are contained in the training data? How many features are contained in a single datum? What is the largest and smalles number contained in the training data?

In [ ]:
# STUDENT CODE HERE

Look at the shape and contents of the training labels. Given the number of pieces of training data that we have and the number of classes amongst them, how does the training labels indicate the class-label for each datum?

This label-formatting is known as **one-hot encoding**, which is a stupid name.

#### STUDENT WRITTEN ANSWER HERE

Use the method provided by `toy_data` to plot the data. What objects are returned by this method? They should be familiar to you in the context of the matplotlib function `plt.subplots()`.

In [ ]:
# STUDENT CODE HERE

Use this plot, and your access to the training data and labels to figure our which color belongs to which class-label (0, 1, or 2).

STUDENT WRTTEN ANSWER HERE
 - red: ????
 - yellow: ????
 - blue: ????



#### Initializing Your Model's Trainable Parameters
Recall the form of our model:
 - $f(X; W, b) = XW + b$
 
 Given the shape of our training data, what are the shapes of $W$ and $b$?
 
Read the documentation for `np.random.randn` (use shift-tab!). What kind of statistical distribution does this draw from? Is it a uniform distribution? Use this to initialize $W$.

Initialize $b$ as zeros?

Should these be numpy arrays or mygrad tensors?

In [ ]:
# STUDENT CODE HERE

#### Preparing your optimization scheme
We will be using "vanilla" gradient descent (referred to as stochastic gradient descent (SGD) in literature) to train our model. Write a simple function that takes in a trainable parameter and any other parameters needed for performing gradient descent. This function should return the updated **data** of the parameter. All operations involving a `Tensor` should only be using its `grad` and `data` properties. We do not want to create a computational graph when we already have the gradients we want! 

This is a rare instance where we want the function to **mutate** the input data: it should work with a *view* of the data and update it. Thus it need not return anything!

Include a reasonably-detailed docstring for this function. Indicate what variable-types it expects and returns.

In [ ]:
# STUDENT CODE HERE

Write a function that takes in your model's output and any other parameters it needs to compute the mean accuracy of your prediction. Write a descent docstring.

In [ ]:
# STUDENT CODE HERE

Import `dense` from mygrad's layers and `multiclass_hinge` from the losses.

In [ ]:
from mygrad.nnet.layers import dense
from mygrad.nnet.losses import multiclass_hinge

Read the documentation for multiclass hinge. What format of `y_train` does it expect? What is the form of our labels? How can you get labels in the form expected by multiclass hinge?

In [ ]:
# STUDENT CODE HERE

Using a learning rate of `1.` and **no** regularization, train your model for 1000 iterations. Record the loss and accuracy for each operation. Plot them afterwards.

In [ ]:
# STUDENT CODE HERE

Look at the values in W. Specifically note the columns. Keeping in mind what columns correspond to which color, visualize where the columns of W would be plotted if they were points. Why does this make sense? Plot the points for further clarification and practice. 

In [ ]:
# STUDENT CODE/ANSWER HERE